# Example

In [14]:
!python3 -m pip install git+git://github.com/tensortrade-org/tensortrade.git

  Cloning git://github.com/tensortrade-org/tensortrade.git to /private/var/folders/r9/1ph6hh194qj_w8209my99snc0000gn/T/pip-req-build-wu0s93p0
  Running command git clone -q git://github.com/tensortrade-org/tensortrade.git /private/var/folders/r9/1ph6hh194qj_w8209my99snc0000gn/T/pip-req-build-wu0s93p0
  Created wheel for tensortrade: filename=tensortrade-0.1.0rc1-cp37-none-any.whl size=79410 sha256=e9288cb7882925c90f42a7426e59bcfd7380f77ec50651d4f420b726338d7aef
  Stored in directory: /private/var/folders/r9/1ph6hh194qj_w8209my99snc0000gn/T/pip-ephem-wheel-cache-_vho_1jn/wheels/d4/5f/92/18e058e7f31d59deb14af7ecd3f907093cc894226b1c7abaa3
Successfully built tensortrade


In [15]:
import ssl
import pandas as pd


ssl._create_default_https_context = ssl._create_unverified_context # Only used if pandas gives a SSLError


def fetch(exchange_name, symbol, timeframe):
    url = "https://www.cryptodatadownload.com/cdd/"
    filename = "{}_{}USD_{}".format(exchange_name, symbol, timeframe)
    df = pd.read_csv('https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv', skiprows=1)
    df = df[::-1]
    df = df.drop(["Symbol", "Volume BTC"], axis=1)
    df = df.rename({"Volume USD": "volume"}, axis=1)
    df.columns = [name.lower() for name in df.columns]
    df = df.set_index("date")
    df.columns = [symbol + ":" + name.lower() for name in df.columns]
    return df

## Historical Data

In [16]:
coinbase_data = pd.concat([
    fetch("Coinbase", "BTC", "1h"),
    fetch("Coinbase", "ETH", "1h")
], axis=1)

bitstamp_data = pd.concat([
    fetch("Bitstamp", "BTC", "1h"),
    fetch("Bitstamp", "ETH", "1h"),
    fetch("Bitstamp", "LTC", "1h")
], axis=1)

In [17]:
coinbase_data.head()

,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume,ETH:open,ETH:high,ETH:low,ETH:close,ETH:volume
date,,,,,,,,,,
2017-07-01 11-AM,2505.56,2513.38,2495.12,2509.17,287000.32,2505.56,2513.38,2495.12,2509.17,287000.32
2017-07-01 12-PM,2509.17,2512.87,2484.99,2488.43,393142.50,2509.17,2512.87,2484.99,2488.43,393142.50
2017-07-01 01-PM,2488.43,2488.43,2454.40,2454.43,693254.01,2488.43,2488.43,2454.40,2454.43,693254.01
2017-07-01 02-PM,2454.43,2473.93,2450.83,2459.35,712864.80,2454.43,2473.93,2450.83,2459.35,712864.80
2017-07-01 03-PM,2459.35,2475.00,2450.00,2467.83,682105.41,2459.35,2475.00,2450.00,2467.83,682105.41


In [18]:
bitstamp_data.head()

,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume,ETH:open,ETH:high,ETH:low,ETH:close,ETH:volume,LTC:open,LTC:high,LTC:low,LTC:close,LTC:volume
date,,,,,,,,,,,,,,,
2017-07-01 11-AM,2505.56,2513.38,2495.12,2509.17,287000.32,2505.56,2513.38,2495.12,2509.17,287000.32,2505.56,2513.38,2495.12,2509.17,287000.32
2017-07-01 12-PM,2509.17,2512.87,2484.99,2488.43,393142.50,2509.17,2512.87,2484.99,2488.43,393142.50,2509.17,2512.87,2484.99,2488.43,393142.50
2017-07-01 01-PM,2488.43,2488.43,2454.40,2454.43,693254.01,2488.43,2488.43,2454.40,2454.43,693254.01,2488.43,2488.43,2454.40,2454.43,693254.01
2017-07-01 02-PM,2454.43,2473.93,2450.83,2459.35,712864.80,2454.43,2473.93,2450.83,2459.35,712864.80,2454.43,2473.93,2450.83,2459.35,712864.80
2017-07-01 03-PM,2459.35,2475.00,2450.00,2467.83,682105.41,2459.35,2475.00,2450.00,2467.83,682105.41,2459.35,2475.00,2450.00,2467.83,682105.41


## Define Exchanges

An exchange needs a name, an execution service, and streams of price data in order to function properly.

The setups supported right now are the simulated execution service using simulated or stochastic data. More execution services will be made available in the future, as well as price streams so that live data and execution can be supported.

In [ ]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream

coinbase = Exchange("coinbase", service=execute_order)(
    Stream("USD-BTC", list(coinbase['BTC:close'])),
    Stream("USD-ETH", list(coinbase['ETH:close']))
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream("USD-BTC", list(bitstamp['BTC:close'])),
    Stream("USD-ETH", list(bitstamp['ETH:close'])),
    Stream("USD-LTC", list(bitstamp['LTC:close']))
)

Now that the exchanges have been defined we can define our features that we would like to include, excluding the prices we have provided for the exchanges.

## Define External Data Feed

Here we will define the external feed to use whatever data you would like. From financial indicators to datasets that have nothing to do with instrument prices, they will all have to be defined and incorporated into the external data feed provided to the environment. There is also an internal data feed that is used to collect data on all the wallets and the net worth of the portfolio. You can choose to include this or not by using the `use_internal` parameter of the trading environment.

In [ ]:
import ta

from tensortrade.data import DataFeed, Namespace

# Add all features for coinbase bitcoin
coinbase_btc = coinbase_data.loc[:, [name.startswith("BTC") for name in coinbase_data.columns]]

ta.add_all_ta_features(
    coinbase_btc,
    colprefix="BTC:",
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

nodes = []
for name in coinbase_btc.columns:
    nodes += [Stream(name, list(coinbase_btc[name]))]

# Namespaces are used to give context to features so they don't 
# conflict with features of the same name that are being used for
# different data (e.g. different exchange)
coinbase_ns = Namespace("coinbase")(*nodes) 
    

# Add all features for coinbase ethereum
bitstamp_eth = bitstamp_data.loc[:, [name.startswith("ETH") for name in bitstamp_data.columns]]  
ta.add_all_ta_features(
    bitstamp_eth,
    colprefix="ETH:",
    **{k: "ETH:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

nodes = []
for name in coinbase_btc.columns:
    nodes += [Stream(name, list(coinbase_btc[name]))]
    
bitstamp_ns = Namespace("coinbase")(*nodes)

feed = DataFeed([coinbase_ns, bitstamp_ns])

In [21]:
feed.next()

{'coinbase:/BTC:open': 2505.56,
 'coinbase:/BTC:high': 2513.38,
 'coinbase:/BTC:low': 2495.12,
 'coinbase:/BTC:close': 2509.17,
 'coinbase:/BTC:volume': 287000.32,
 'coinbase:/BTC:volume_adi': 154659.5371741516,
 'coinbase:/BTC:volume_obv': 287000.32,
 'coinbase:/BTC:volume_cmf': 0.5388828039430464,
 'coinbase:/BTC:volume_fi': nan,
 'coinbase:/BTC:volume_em': nan,
 'coinbase:/BTC:volume_sma_em': nan,
 'coinbase:/BTC:volume_vpt': -187654.4252942016,
 'coinbase:/BTC:volume_nvi': 1000.0,
 'coinbase:/BTC:volatility_atr': 0.0,
 'coinbase:/BTC:volatility_bbm': 2509.17,
 'coinbase:/BTC:volatility_bbh': 2509.17,
 'coinbase:/BTC:volatility_bbl': 2509.17,
 'coinbase:/BTC:volatility_bbw': 0.0,
 'coinbase:/BTC:volatility_bbhi': 0.0,
 'coinbase:/BTC:volatility_bbli': 0.0,
 'coinbase:/BTC:volatility_kcc': 2505.89,
 'coinbase:/BTC:volatility_kch': 2505.89,
 'coinbase:/BTC:volatility_kcl': 2487.6299999999997,
 'coinbase:/BTC:volatility_kchi': 0.0,
 'coinbase:/BTC:volatility_kcli': 0.0,
 'coinbase:/BTC

## Portfolio

Make the portfolio using the any combinations of exchanges and intruments that the exchange supports

In [24]:
from tensortrade.instruments import USD, BTC, ETH, LTC
from tensortrade.wallets import Wallet, Portfolio

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC),
    Wallet(coinbase, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 20 * ETH),
    Wallet(bitstamp, 3 * LTC),
])

## Environment

In [29]:
from tensortrade.environments import TradingEnvironment

env = TradingEnvironment(
    portfolio=portfolio,
    action_scheme='managed-risk',
    reward_scheme='simple',
    feed=feed,
    window_size=15,
    enable_logger=False
)

In [30]:
env.feed.next()

{'coinbase:/BTC:open': 2505.56,
 'coinbase:/BTC:high': 2513.38,
 'coinbase:/BTC:low': 2495.12,
 'coinbase:/BTC:close': 2509.17,
 'coinbase:/BTC:volume': 287000.32,
 'coinbase:/BTC:volume_adi': 154659.5371741516,
 'coinbase:/BTC:volume_obv': 287000.32,
 'coinbase:/BTC:volume_cmf': 0.5388828039430464,
 'coinbase:/BTC:volume_fi': nan,
 'coinbase:/BTC:volume_em': nan,
 'coinbase:/BTC:volume_sma_em': nan,
 'coinbase:/BTC:volume_vpt': -187654.4252942016,
 'coinbase:/BTC:volume_nvi': 1000.0,
 'coinbase:/BTC:volatility_atr': 0.0,
 'coinbase:/BTC:volatility_bbm': 2509.17,
 'coinbase:/BTC:volatility_bbh': 2509.17,
 'coinbase:/BTC:volatility_bbl': 2509.17,
 'coinbase:/BTC:volatility_bbw': 0.0,
 'coinbase:/BTC:volatility_bbhi': 0.0,
 'coinbase:/BTC:volatility_bbli': 0.0,
 'coinbase:/BTC:volatility_kcc': 2505.89,
 'coinbase:/BTC:volatility_kch': 2505.89,
 'coinbase:/BTC:volatility_kcl': 2487.6299999999997,
 'coinbase:/BTC:volatility_kchi': 0.0,
 'coinbase:/BTC:volatility_kcli': 0.0,
 'coinbase:/BTC